### Format and combine frame txt files of yolo_traking 
path should be specified accordingly

#### MOT17

In [ ]:
import os
import re
from PIL import Image
from tqdm import tqdm

def get_image_dimensions(image_directory):
    example_image_path = os.path.join(image_directory, os.listdir(image_directory)[0])
    with Image.open(example_image_path) as img:
        width, height = img.size
    return width, height

def process_file(file_path, image_width, image_height):
    frame_number = int(os.path.basename(file_path).split('_')[-1].split('.')[0])
    output_data = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines
        if not lines:
            print(f"File {file_path} is empty.")
        for line in lines:
            data = line.strip().split()
            if len(data) == 6:
                track_id = data[-1]
                bbox_data = data[1:5]  # Ignore the first column
                x_center, y_center, width, height = map(float, bbox_data)
                
                # Convert normalized coordinates to pixel values
                x_center *= image_width
                y_center *= image_height
                width *= image_width
                height *= image_height
                
                # Convert center coordinates to top-left coordinates
                x_top_left = x_center - (width / 2)
                y_top_left = y_center - (height / 2)
                
                output_line = f"{frame_number},{track_id},{x_top_left:.2f},{y_top_left:.2f},{width:.2f},{height:.2f},1,-1,-1,-1"
                output_data.append(output_line)
    
    return output_data

def sorted_numerically(file_list):
    def numerical_sort(value):
        numbers = re.findall(r'\d+', value)
        return list(map(int, numbers)) if numbers else [0]
    return sorted(file_list, key=numerical_sort)

def process_video(video_name, input_directory, image_directory, output_directory):
    # Get the dimensions of one image
    image_width, image_height = get_image_dimensions(image_directory)
    
    output_file = os.path.join(output_directory, f"{video_name}.txt")
    
    label_directory = os.path.join(input_directory, 'labels')
    files = [file_name for file_name in os.listdir(label_directory) if file_name.endswith('.txt')]
    files = sorted_numerically(files)
    
    all_data = []
    
    for file_name in files:
        file_path = os.path.join(label_directory, file_name)
        #print(f"Processing file: {file_path}")
        output_data = process_file(file_path, image_width, image_height)
        if not output_data:
            print(f"No data processed for file: {file_path}")
        all_data.extend(output_data)
    
    # Sort all data by frame number first, then by track ID
    all_data.sort(key=lambda x: (int(x.split(',')[0]), int(x.split(',')[1])))
    
    with open(output_file, 'w') as out_file:
        for line in all_data:
            out_file.write(line + '\n')
    
    print(f"Data has been written to {output_file}")

def main(dataset_dir, result_base_dir, output_base_dir):
    trackers = ['deepocsort', 'botsort']
    # trackers = ['botsort']
    
    for tracker in trackers:
        result_dir = os.path.join(result_base_dir, f"{tracker}__input_1280__conf_.25")
        output_dir = os.path.join(output_base_dir, f"{tracker.upper()}__input_1280__conf_.25", "data")
        
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        video_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        
        for video_dir in tqdm(video_dirs, desc=f"Processing videos for {tracker}", unit="video"):
            video_name = video_dir
            input_directory = os.path.join(result_dir, video_name)
            image_directory = os.path.join(dataset_dir, video_name, 'img1')
            process_video(video_name, input_directory, image_directory, output_dir)

# Specify the dataset directory, base result directory, and base output directory
dataset_dir = '/media/hbai/data/code/LiteSORT/datasets/MOT17/train'
result_base_dir = '/media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/MOT17-train_LITE'
output_base_dir = '/media/hbai/data/code/LiteSORT/results/off-the-shelf/MOT17-train'
# Run the processing for all videos in the dataset for all trackers
main(dataset_dir, result_base_dir, output_base_dir)


#### MOT20

In [ ]:
import os
import re
from PIL import Image
from tqdm import tqdm

def get_image_dimensions(image_directory):
    example_image_path = os.path.join(image_directory, os.listdir(image_directory)[0])
    with Image.open(example_image_path) as img:
        width, height = img.size
    return width, height

def process_file(file_path, image_width, image_height):
    frame_number = int(os.path.basename(file_path).split('_')[-1].split('.')[0])
    output_data = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines
        # print(f'lines: {lines}')
        if not lines:
            print(f"File {file_path} is empty.")
        for line in lines:
            data = line.strip().split()
            # print(f'data: {data}')
            if len(data) == 6:
                track_id = data[-1]
                bbox_data = data[1:5]  # Ignore the first column
                x_center, y_center, width, height = map(float, bbox_data)
                
                # Convert normalized coordinates to pixel values
                x_center *= image_width
                y_center *= image_height
                width *= image_width
                height *= image_height
                
                # Convert center coordinates to top-left coordinates
                x_top_left = x_center - (width / 2)
                y_top_left = y_center - (height / 2)
                
                output_line = f"{frame_number},{track_id},{x_top_left:.2f},{y_top_left:.2f},{width:.2f},{height:.2f},1,-1,-1,-1"
                # print(f'output_line: {output_line}')
                output_data.append(output_line)
    
    return output_data



# def sorted_numerically(file_list):
#     def numerical_sort(value):
#         numbers = re.findall(r'\d+', value)
#         return list(map(int, numbers)) if numbers else [0]
#     return sorted(file_list, key=numerical_sort)

def process_video(seq_name, input_directory, output_directory):
    
    output_file = os.path.join(output_directory, f"{seq_name}.txt")
    # print(f'output_file: {output_file}')
    
    label_directory = os.path.join(input_directory, 'labels')
    files = [file_name for file_name in os.listdir(label_directory) if file_name.endswith('.txt')]
    # files = sorted_numerically(files)
    # print(f'files: {files}')
    all_data = []
    
    for file_name in files:
        file_path = os.path.join(label_directory, file_name)
        print(f"Processing file: {file_path}")
        output_data = process_file(file_path)
        if not output_data:
            print(f"No data processed for file: {file_path}")
        all_data.extend(output_data)
    
    # Sort all data by frame number first, then by track ID
    all_data.sort(key=lambda x: (int(x.split(',')[0]), int(x.split(',')[1])))
    
    with open(output_file, 'w') as out_file:
        for line in all_data:
            out_file.write(line + '\n')
    
    print(f"Data has been written to {output_file}")



def main(dataset_dir, result_base_dir, output_base_dir):
    # trackers = ['deepocsort', 'botsort', 'ocsort', 'bytetrack']
    trackers = ['deepocsort', 'botsort']
    
    for tracker in trackers:
        result_dir = os.path.join(result_base_dir, f"{tracker}__input_1280__conf_.25")
        output_dir = os.path.join(output_base_dir, f"{tracker.upper()}__input_1280__conf_.25", "data")
        
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        video_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        
        for video_dir in tqdm(video_dirs, desc=f"Processing videos for {tracker}", unit="video"):
            video_name = video_dir
            input_directory = os.path.join(result_dir, video_name)
            image_directory = os.path.join(dataset_dir, video_name, 'img1')
            process_video(video_name, input_directory, image_directory, output_dir)

# Specify the dataset directory, base result directory, and base output directory
dataset_dir = '/media/hbai/data/code/LiteSORT/datasets/MOT20/train'
result_base_dir = '/media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/MOT20-train_LITE'
output_base_dir = '/media/hbai/data/code/LiteSORT/results/off-the-shelf/MOT20-train'

# Run the processing for all videos in the dataset for all trackers
main(dataset_dir, result_base_dir, output_base_dir)


#### PersonPath22

In [ ]:
import os
import re
from PIL import Image
from tqdm import tqdm

def get_image_dimensions(image_directory):
    image_files = os.listdir(image_directory)
    if not image_files:
        raise FileNotFoundError(f"No images found in directory: {image_directory}")
    
    example_image_path = os.path.join(image_directory, image_files[0])
    with Image.open(example_image_path) as img:
        width, height = img.size
    return width, height

def process_file(file_path, image_width, image_height):
    frame_number = int(os.path.basename(file_path).split('_')[-1].split('.')[0]) + 1
    
    output_data = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines
        if not lines:
            print(f"File {file_path} is empty.")
        for line in lines:
            data = line.strip().split()
            if len(data) == 6:
                try:
                    track_id = data[-1]
                    bbox_data = data[1:5]  # Ignore the first column
                    
                    x_center, y_center, width, height = map(float, bbox_data)
                    
                    # Convert normalized coordinates to pixel values
                    x_center *= image_width
                    y_center *= image_height
                    width *= image_width
                    height *= image_height
                    
                    # Convert center coordinates to top-left coordinates
                    x_top_left = x_center - (width / 2)
                    y_top_left = y_center - (height / 2)
                    
                    output_line = f"{frame_number},{track_id},{x_top_left:.2f},{y_top_left:.2f},{width:.2f},{height:.2f},1,-1,-1,-1"
                    output_data.append(output_line)
                except ValueError as e:
                    print(f"Error processing line in file {file_path}: {line} -> {e}")
    
    return output_data

def sorted_numerically(file_list):
    def numerical_sort(value):
        numbers = re.findall(r'\d+', value)
        return list(map(int, numbers)) if numbers else [0]
    return sorted(file_list, key=numerical_sort)

def process_video(video_name, input_directory, image_directory, output_directory):
    try:
        # Get the dimensions of one image
        image_width, image_height = get_image_dimensions(image_directory)
    except FileNotFoundError as e:
        print(e)
        return
    
    output_file = os.path.join(output_directory, f"{video_name}.txt")
    
    label_directory = os.path.join(input_directory, 'labels')
    if not os.path.exists(label_directory):
        print(f"Label directory does not exist: {label_directory}")
        return
    
    files = [file_name for file_name in os.listdir(label_directory) if file_name.endswith('.txt')]
    if not files:
        print(f"No label files found in directory: {label_directory}")
        return
    
    files = sorted_numerically(files)
    
    all_data = []
    
    for file_name in files:
        file_path = os.path.join(label_directory, file_name)
        output_data = process_file(file_path, image_width, image_height)
        if not output_data:
            print(f"No data processed for file: {file_path}")
        all_data.extend(output_data)
    
    if not all_data:
        print(f"No data to write for video: {video_name}")
        return
    
    # Sort all data by frame number first, then by track ID
    all_data.sort(key=lambda x: (int(x.split(',')[0]), int(x.split(',')[1])))
    
    with open(output_file, 'w') as out_file:
        for line in all_data:
            out_file.write(line + '\n')
    
    print(f"Data has been written to {output_file}")

def main(dataset_dir, result_base_dir, output_base_dir):
    trackers = ['botsort']
    
    for tracker in trackers:
        result_dir = os.path.join(result_base_dir, f"{tracker}__input_1280__conf_.25")
        output_dir = os.path.join(output_base_dir, f"{tracker.upper()}__input_1280__conf_.25", "data")
        
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        video_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        
        for video_dir in tqdm(video_dirs, desc=f"Processing videos for {tracker}", unit="video"):
            video_name = video_dir
            input_directory = os.path.join(result_dir, video_name)
            image_directory = os.path.join(dataset_dir, video_name, 'img1')
            process_video(video_name, input_directory, image_directory, output_dir)

# Specify the dataset directory, base result directory, and base output directory
dataset_dir = '/media/hbai/data/code/LiteSORT/datasets/PersonPath22/test'
result_base_dir = '/media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/PersonPath22_LITE'
output_base_dir = '/media/hbai/data/code/LiteSORT/results/off-the-shelf/person_path_22-test'

# Run the processing for all videos in the dataset for all trackers
main(dataset_dir, result_base_dir, output_base_dir)


#### VIRAT-S

In [ ]:
import os
import re
from PIL import Image
from tqdm import tqdm

def get_image_dimensions(image_directory):
    image_files = os.listdir(image_directory)
    if not image_files:
        raise FileNotFoundError(f"No images found in directory: {image_directory}")
    
    example_image_path = os.path.join(image_directory, image_files[0])
    with Image.open(example_image_path) as img:
        width, height = img.size
    return width, height

def process_file(file_path, image_width, image_height):
    frame_number = int(os.path.basename(file_path).split('_')[-1].split('.')[0]) + 1
    
    output_data = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines
        if not lines:
            print(f"File {file_path} is empty.")
        for line in lines:
            data = line.strip().split()
            if len(data) == 6:
                try:
                    track_id = data[-1]
                    bbox_data = data[1:5]  # Ignore the first column
                    
                    x_center, y_center, width, height = map(float, bbox_data)
                    
                    # Convert normalized coordinates to pixel values
                    x_center *= image_width
                    y_center *= image_height
                    width *= image_width
                    height *= image_height
                    
                    # Convert center coordinates to top-left coordinates
                    x_top_left = x_center - (width / 2)
                    y_top_left = y_center - (height / 2)
                    
                    output_line = f"{frame_number - 1},{track_id},{x_top_left:.2f},{y_top_left:.2f},{width:.2f},{height:.2f},1,-1,-1,-1"
                    output_data.append(output_line)
                except ValueError as e:
                    print(f"Error processing line in file {file_path}: {line} -> {e}")
    
    return output_data

def sorted_numerically(file_list):
    def numerical_sort(value):
        numbers = re.findall(r'\d+', value)
        return list(map(int, numbers)) if numbers else [0]
    return sorted(file_list, key=numerical_sort)

def process_video(video_name, input_directory, image_directory, output_directory):
    try:
        # Get the dimensions of one image
        image_width, image_height = get_image_dimensions(image_directory)
    except FileNotFoundError as e:
        print(e)
        return
    
    output_file = os.path.join(output_directory, f"{video_name}.txt")
    
    label_directory = os.path.join(input_directory, 'labels')
    if not os.path.exists(label_directory):
        print(f"Label directory does not exist: {label_directory}")
        return
    
    files = [file_name for file_name in os.listdir(label_directory) if file_name.endswith('.txt')]
    if not files:
        print(f"No label files found in directory: {label_directory}")
        return
    
    files = sorted_numerically(files)
    
    all_data = []
    
    for file_name in files:
        file_path = os.path.join(label_directory, file_name)
        output_data = process_file(file_path, image_width, image_height)
        if not output_data:
            print(f"No data processed for file: {file_path}")
        all_data.extend(output_data)
    
    if not all_data:
        print(f"No data to write for video: {video_name}")
        return
    
    # Sort all data by frame number first, then by track ID
    all_data.sort(key=lambda x: (int(x.split(',')[0]), int(x.split(',')[1])))
    
    with open(output_file, 'w') as out_file:
        for line in all_data:
            out_file.write(line + '\n')
    
    print(f"Data has been written to {output_file}")

def main(dataset_dir, result_base_dir, output_base_dir):
    trackers = ['deepocsort', 'botsort', 'ocsort', 'bytetrack']
    
    for tracker in trackers:
        result_dir = os.path.join(result_base_dir, f"{tracker}__input_1280__conf_.25")
        output_dir = os.path.join(output_base_dir, f"{tracker.upper()}__input_1280__conf_.25", "data")
        
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        video_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        
        for video_dir in tqdm(video_dirs, desc=f"Processing videos for {tracker}", unit="video"):
            video_name = video_dir
            input_directory = os.path.join(result_dir, video_name)
            image_directory = os.path.join(dataset_dir, video_name, 'img1')
            process_video(video_name, input_directory, image_directory, output_dir)

# Specify the dataset directory, base result directory, and base output directory
dataset_dir = '/media/hbai/data/code/LiteSORT/datasets/VIRAT-S/train'
result_base_dir = '/media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/VIRAT-S_LITE'
output_base_dir = '/media/hbai/data/code/LiteSORT/results/off-the-shelf/VIRAT-S_LITE'

# Run the processing for all videos in the dataset for all trackers
main(dataset_dir, result_base_dir, output_base_dir)


#### KITTI

to convert from yolo to MOT format

In [ ]:
import os
import re
from PIL import Image
from tqdm import tqdm

def get_image_dimensions(image_directory):
    image_files = os.listdir(image_directory)
    if not image_files:
        raise FileNotFoundError(f"No images found in directory: {image_directory}")
    
    example_image_path = os.path.join(image_directory, image_files[0])
    with Image.open(example_image_path) as img:
        width, height = img.size
    return width, height

def process_file(file_path, image_width, image_height):
    frame_number = int(os.path.basename(file_path).split('_')[-1].split('.')[0]) + 1
    
    output_data = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines
        if not lines:
            print(f"File {file_path} is empty.")
        for line in lines:
            data = line.strip().split()
            if len(data) == 6:
                try:
                    track_id = data[-1]
                    bbox_data = data[1:5]  # Ignore the first column
                    
                    x_center, y_center, width, height = map(float, bbox_data)
                    
                    # Convert normalized coordinates to pixel values
                    x_center *= image_width
                    y_center *= image_height
                    width *= image_width
                    height *= image_height
                    
                    # Convert center coordinates to top-left coordinates
                    x_top_left = x_center - (width / 2)
                    y_top_left = y_center - (height / 2)
                    
                    output_line = f"{frame_number - 1},{track_id},{x_top_left:.2f},{y_top_left:.2f},{width:.2f},{height:.2f},1,-1,-1,-1"
                    output_data.append(output_line)
                except ValueError as e:
                    print(f"Error processing line in file {file_path}: {line} -> {e}")
    
    return output_data

def sorted_numerically(file_list):
    def numerical_sort(value):
        numbers = re.findall(r'\d+', value)
        return list(map(int, numbers)) if numbers else [0]
    return sorted(file_list, key=numerical_sort)

def process_video(video_name, input_directory, image_directory, output_directory):
    try:
        # Get the dimensions of one image
        image_width, image_height = get_image_dimensions(image_directory)
    except FileNotFoundError as e:
        print(e)
        return
    
    output_file = os.path.join(output_directory, f"{video_name}.txt")
    
    label_directory = os.path.join(input_directory, 'labels')
    if not os.path.exists(label_directory):
        print(f"Label directory does not exist: {label_directory}")
        return
    
    files = [file_name for file_name in os.listdir(label_directory) if file_name.endswith('.txt')]
    if not files:
        print(f"No label files found in directory: {label_directory}")
        return
    
    files = sorted_numerically(files)
    
    all_data = []
    
    for file_name in files:
        file_path = os.path.join(label_directory, file_name)
        output_data = process_file(file_path, image_width, image_height)
        if not output_data:
            print(f"No data processed for file: {file_path}")
        all_data.extend(output_data)
    
    if not all_data:
        print(f"No data to write for video: {video_name}")
        return
    
    # Sort all data by frame number first, then by track ID
    all_data.sort(key=lambda x: (int(x.split(',')[0]), int(x.split(',')[1])))
    
    with open(output_file, 'w') as out_file:
        for line in all_data:
            out_file.write(line + '\n')
    
    print(f"Data has been written to {output_file}")

def main(dataset_dir, result_base_dir, output_base_dir):
    # trackers = ['deepocsort', 'botsort', 'ocsort', 'bytetrack']
    trackers = ['deepocsort']
    
    for tracker in trackers:
        result_dir = os.path.join(result_base_dir, f"{tracker}__input_1280__conf_.25")
        output_dir = os.path.join(output_base_dir, f"{tracker.upper()}__input_1280__conf_.25", "data")
        
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        video_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        
        for video_dir in tqdm(video_dirs, desc=f"Processing videos for {tracker}", unit="video"):
            video_name = video_dir
            input_directory = os.path.join(result_dir, video_name)
            image_directory = os.path.join(dataset_dir, video_name, 'img1')
            process_video(video_name, input_directory, image_directory, output_dir)

# Specify the dataset directory, base result directory, and base output directory
dataset_dir = '/media/hbai/data/code/LiteSORT/datasets/KITTI/train'
result_base_dir = '/media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/KITTI_LITE'
output_base_dir = '/media/hbai/data/code/LiteSORT/results/off-the-shelf/KITTI-train'

# Run the processing for all videos in the dataset for all trackers
main(dataset_dir, result_base_dir, output_base_dir)


to convert from MOT to KITTI format

In [ ]:
# to convert from MOT to KITTI format
import os

def mot_to_kitti(input_file, output_file):
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        for line in f_in:
            # Parse the input line
            fields = line.strip().split(',')
            
            # Extract MOT fields
            frame = int(fields[0])
            track_id = int(fields[1])
            bbox_left = float(fields[2])
            bbox_top = float(fields[3])
            bbox_width = float(fields[4])
            bbox_height = float(fields[5])
            score = float(fields[6])
            
            # Set default values for KITTI fields
            object_type = "pedestrian"  # default to pedestrian
            truncated = -1
            occluded = -1
            alpha = -10
            dimensions = (-1, -1, -1)
            location = (-1000, -1000, -1000)
            rotation_y = -10

            # Compute the right and bottom coordinates
            bbox_right = bbox_left + bbox_width
            bbox_bottom = bbox_top + bbox_height
            
            # Write to the output file in KITTI format
            f_out.write(f"{frame} {track_id} {object_type} {truncated} {occluded} {alpha:.2f} "
                        f"{bbox_left:.2f} {bbox_top:.2f} {bbox_right:.2f} {bbox_bottom:.2f} "
                        f"{location[0]:.2f} {location[1]:.2f} {location[2]:.2f} "
                        f"{dimensions[0]:.2f} {dimensions[1]:.2f} {dimensions[2]:.2f} "
                        f"{rotation_y:.2f} {score:.2f}\n")

def process_directory(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.txt'):
            input_file = os.path.join(input_dir, file_name)
            output_file = os.path.join(output_dir, file_name)
            mot_to_kitti(input_file, output_file)
            print(f"Converted {input_file} to {output_file}")

def process_all_trackers(base_input_dir, base_output_dir, trackers):
    for tracker in trackers:
        input_dir = os.path.join(base_input_dir, tracker, 'data')
        output_dir = os.path.join(base_output_dir, f"{tracker}", 'data')
        print(f"Processing tracker: {tracker}")
        process_directory(input_dir, output_dir)

# Usage example
base_input_dir = '/media/hbai/data/code/LiteSORT/results/off-the-shelf/KITTI-train'
base_output_dir = '/media/hbai/data/code/LiteSORT/results/off-the-shelf/KITTI-train_converted'
trackers = [
    'BoTSORT__input_1280__conf_.25',
    # 'ByteTRACK__input_1280__conf_.25',
    'DeepOC-SORT__input_1280__conf_.25',
    # 'OCSORT__input_1280__conf_.25'
]

process_all_trackers(base_input_dir, base_output_dir, trackers)



#### DanceTrack 

In [ ]:
import os
import re
from PIL import Image
from tqdm import tqdm

def get_image_dimensions(image_directory):
    image_files = os.listdir(image_directory)
    if not image_files:
        raise FileNotFoundError(f"No images found in directory: {image_directory}")
    
    example_image_path = os.path.join(image_directory, image_files[0])
    with Image.open(example_image_path) as img:
        width, height = img.size
    return width, height

def process_file(file_path, image_width, image_height):
    frame_number = int(os.path.basename(file_path).split('_')[-1].split('.')[0])
    
    output_data = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines
        if not lines:
            print(f"File {file_path} is empty.")
        for line in lines:
            data = line.strip().split()
            if len(data) == 6:
                try:
                    track_id = data[-1]
                    bbox_data = data[1:5]  # Ignore the first column
                    
                    x_center, y_center, width, height = map(float, bbox_data)
                    
                    # Convert normalized coordinates to pixel values
                    x_center *= image_width
                    y_center *= image_height
                    width *= image_width
                    height *= image_height
                    
                    # Convert center coordinates to top-left coordinates
                    x_top_left = x_center - (width / 2)
                    y_top_left = y_center - (height / 2)
                    
                    output_line = f"{frame_number},{track_id},{x_top_left:.2f},{y_top_left:.2f},{width:.2f},{height:.2f},1,-1,-1,-1"
                    output_data.append(output_line)
                except ValueError as e:
                    print(f"Error processing line in file {file_path}: {line} -> {e}")
    
    return output_data

def sorted_numerically(file_list):
    def numerical_sort(value):
        numbers = re.findall(r'\d+', value)
        return list(map(int, numbers)) if numbers else [0]
    return sorted(file_list, key=numerical_sort)

def process_video(video_name, input_directory, image_directory, output_directory):
    try:
        # Get the dimensions of one image
        image_width, image_height = get_image_dimensions(image_directory)
    except FileNotFoundError as e:
        print(e)
        return
    
    output_file = os.path.join(output_directory, f"{video_name}.txt")
    
    label_directory = os.path.join(input_directory, 'labels')
    if not os.path.exists(label_directory):
        print(f"Label directory does not exist: {label_directory}")
        return
    
    files = [file_name for file_name in os.listdir(label_directory) if file_name.endswith('.txt')]
    if not files:
        print(f"No label files found in directory: {label_directory}")
        return
    
    files = sorted_numerically(files)
    
    all_data = []
    
    for file_name in files:
        file_path = os.path.join(label_directory, file_name)
        output_data = process_file(file_path, image_width, image_height)
        if not output_data:
            print(f"No data processed for file: {file_path}")
        all_data.extend(output_data)
    
    if not all_data:
        print(f"No data to write for video: {video_name}")
        return
    
    # Sort all data by frame number first, then by track ID
    all_data.sort(key=lambda x: (int(x.split(',')[0]), int(x.split(',')[1])))
    
    with open(output_file, 'w') as out_file:
        for line in all_data:
            out_file.write(line + '\n')
    
    print(f"Data has been written to {output_file}")

def main(dataset_dir, result_base_dir, output_base_dir):
    trackers = ['lite_botsort', 'lite_deepocsort']
    
    for tracker in trackers:
        result_dir = os.path.join(result_base_dir, f"{tracker}__input_1280__conf_.25")
        output_dir = os.path.join(output_base_dir, f"{tracker.upper()}__input_1280__conf_.25", "data")
        
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        video_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        
        for video_dir in tqdm(video_dirs, desc=f"Processing videos for {tracker}", unit="video"):
            video_name = video_dir
            input_directory = os.path.join(result_dir, video_name)
            image_directory = os.path.join(dataset_dir, video_name, 'img1')
            process_video(video_name, input_directory, image_directory, output_dir)

# Specify the dataset directory, base result directory, and base output directory
dataset_dir = '/media/hbai/data/code/LiteSORT/datasets/DanceTrack/train'
result_base_dir = '/media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train'
output_base_dir = '/media/hbai/data/code/LiteSORT/results/off-the-shelf/DanceTrack-train'

# Run the processing for all videos in the dataset for all trackers
main(dataset_dir, result_base_dir, output_base_dir)


### Visualize Formated txt
path should be specified accordingly

#### MOT20

In [ ]:
import os
import cv2

def draw_bounding_boxes(image_dir, result_file, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    with open(result_file, 'r') as file:
        lines = file.readlines()
    
    bbox_data = {}
    for line in lines:
        data = line.strip().split(',')
        frame_id = int(data[0])
        track_id = int(data[1])
        x, y, w, h = map(float, data[2:6])
        
        if frame_id not in bbox_data:
            bbox_data[frame_id] = []
        
        bbox_data[frame_id].append((track_id, x, y, w, h))
    
    for frame_id in bbox_data:
        image_path = os.path.join(image_dir, f"{frame_id:06d}.jpg")
        image = cv2.imread(image_path)
        
        for bbox in bbox_data[frame_id]:
            track_id, x, y, w, h = bbox
            top_left = (int(x), int(y))
            bottom_right = (int(x + w), int(y + h))
            cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
            cv2.putText(image, str(track_id), (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        output_image_path = os.path.join(output_dir, f"{frame_id:06d}.jpg")
        cv2.imwrite(output_image_path, image)
        print(f"Processed frame {frame_id}")

# Example usage
image_dir = '/workspace/LiteSORT/datasets/MOT20/train/MOT20-05/img1'  # Adjust this to your image directory
result_file = '/workspace/LiteSORT/results/off-the-shelf/MOT20-train/DeepOC-SORT__input_1280__conf_.25/data/MOT20-05.txt'  # Adjust this to your result file
output_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/viz_MOT20/viz_deepocsort__input_1280__conf_.25/MOT20-05'  # Adjust this to your desired output directory

draw_bounding_boxes(image_dir, result_file, output_dir)


#### PersonPath22

In [ ]:
import os
import cv2

def draw_bounding_boxes(image_dir, result_file, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    with open(result_file, 'r') as file:
        lines = file.readlines()
    
    bbox_data = {}
    for line in lines:
        data = line.strip().split(',')
        frame_id = int(data[0])
        track_id = int(data[1])
        x, y, w, h = map(float, data[2:6])
        
        if frame_id not in bbox_data:
            bbox_data[frame_id] = []
        
        bbox_data[frame_id].append((track_id, x, y, w, h))
    
    for frame_id in bbox_data:
        image_path = os.path.join(image_dir, f"{frame_id:06d}.jpg")
        image = cv2.imread(image_path)
        
        for bbox in bbox_data[frame_id]:
            track_id, x, y, w, h = bbox
            top_left = (int(x), int(y))
            bottom_right = (int(x + w), int(y + h))
            cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
            cv2.putText(image, str(track_id), (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        output_image_path = os.path.join(output_dir, f"{frame_id:06d}.jpg")
        
        cv2.imwrite(output_image_path, image)
        print(f"Processed frame {frame_id}")

# Example usage
image_dir = '/workspace/LiteSORT/datasets/PersonPath22/test/uid_vid_00011.mp4/img1'  # Adjust this to your image directory
result_file = '/workspace/LiteSORT/results/off-the-shelf/PersonPath22/BYTETRACK__input_1280__conf_.25/uid_vid_00011.mp4.txt'  # Adjust this to your result file
output_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/viz_PersonPath22/viz_bytetrack__input_1280_conf_.25/uid_vid_00011'  # Adjust this to your desired output directory

draw_bounding_boxes(image_dir, result_file, output_dir)


In [ ]:
import os
import cv2

def create_video_from_images(image_dir, output_video_path, fps=30):
    # Get a list of all image files in the directory
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
    image_files.sort()  # Ensure the files are sorted in the correct order

    # Read the first image to get the dimensions
    first_image_path = os.path.join(image_dir, image_files[0])
    frame = cv2.imread(first_image_path)
    height, width, layers = frame.shape

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 format
    video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        frame = cv2.imread(image_path)
        video.write(frame)

    # Release the VideoWriter
    video.release()
    print(f"Video saved to {output_video_path}")

# Example usage
image_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/viz_PersonPath/viz_bytetrack__input_1280_conf_.25/uid_vid_00008'  # Adjust this to your image directory
output_video_path = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/viz_PersonPath/viz_bytetrack__input_1280_conf_.25/uid_vid_00008.mp4'  # Adjust this to your desired output video path

create_video_from_images(image_dir, output_video_path)


#### VIRAT-S

In [ ]:
import os
import cv2

def draw_bounding_boxes(image_dir, result_file, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    with open(result_file, 'r') as file:
        lines = file.readlines()
    
    bbox_data = {}
    for line in lines:
        data = line.strip().split(',')
        frame_id = int(data[0])
        track_id = int(data[1])
        x, y, w, h = map(float, data[2:6])
        
        if frame_id not in bbox_data:
            bbox_data[frame_id] = []
        
        bbox_data[frame_id].append((track_id, x, y, w, h))
    
    for frame_id in bbox_data:
        image_path = os.path.join(image_dir, f"{frame_id:06d}.jpg")
        image = cv2.imread(image_path)
        
        for bbox in bbox_data[frame_id]:
            track_id, x, y, w, h = bbox
            top_left = (int(x), int(y))
            bottom_right = (int(x + w), int(y + h))
            cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
            cv2.putText(image, str(track_id), (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        output_image_path = os.path.join(output_dir, f"{frame_id:06d}.jpg")
        
        cv2.imwrite(output_image_path, image)
        print(f"Processed frame {frame_id}")

# Example usage
image_dir = '/workspace/LiteSORT/datasets/VIRAT-S/train/VIRAT_S_010000_06_000728_000762/img1'  # Adjust this to your image directory
result_file = '/workspace/LiteSORT/results/off-the-shelf/VIRAT-S/ByteTRACK__input_1280__conf_.25/data/VIRAT_S_010000_06_000728_000762.txt'  # Adjust this to your result file
output_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/viz_VIRAT-S/viz_ByteTRACK__input_1280__conf_.25/VIRAT_S_010000_06_000728_000762'  # Adjust this to your desired output directory

draw_bounding_boxes(image_dir, result_file, output_dir)


In [ ]:
import os
import re
from PIL import Image
from tqdm import tqdm

def get_image_dimensions(image_directory):
    example_image_path = os.path.join(image_directory, os.listdir(image_directory)[0])
    with Image.open(example_image_path) as img:
        width, height = img.size
    return width, height

def process_file(file_path, image_width, image_height):
    frame_number = os.path.basename(file_path).split('_')[-1].split('.')[0]
    output_data = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines
        if not lines:
            print(f"File {file_path} is empty.")
        for line in lines:
            data = line.strip().split()
            if len(data) == 6:
                track_id = data[-1]
                bbox_data = data[1:5]  # Ignore the first column
                x_center, y_center, width, height = map(float, bbox_data)
                
                # Convert normalized coordinates to pixel values
                x_center *= image_width
                y_center *= image_height
                width *= image_width
                height *= image_height
                
                # Convert center coordinates to top-left coordinates
                x_top_left = x_center - (width / 2)
                y_top_left = y_center - (height / 2)
                
                output_line = f"{frame_number},{track_id},{x_top_left:.2f},{y_top_left:.2f},{width:.2f},{height:.2f},1,-1,-1,-1"
                output_data.append(output_line)
    
    return output_data

def sorted_numerically(file_list):
    def numerical_sort(value):
        numbers = re.findall(r'\d+', value)
        return list(map(int, numbers)) if numbers else [0]
    return sorted(file_list, key=numerical_sort)

def process_video(video_name, input_directory, image_directory, output_directory):
    # Get the dimensions of one image
    image_width, image_height = get_image_dimensions(image_directory)
    
    output_file = os.path.join(output_directory, f"{video_name}.txt")
    
    label_directory = os.path.join(input_directory, 'labels')
    files = [file_name for file_name in os.listdir(label_directory) if file_name.startswith(video_name) and file_name.endswith('.txt')]
    files = sorted_numerically(files)
    
    all_data = []
    
    for file_name in files:
        file_path = os.path.join(label_directory, file_name)
        #print(f"Processing file: {file_path}")
        output_data = process_file(file_path, image_width, image_height)
        if not output_data:
            print(f"No data processed for file: {file_path}")
        all_data.extend(output_data)
    
    # Sort all data by frame number first, then by track ID
    all_data.sort(key=lambda x: (int(x.split(',')[0]), int(x.split(',')[1])))
    
    with open(output_file, 'w') as out_file:
        for line in all_data:
            out_file.write(line + '\n')
    
    print(f"Data has been written to {output_file}")

def main(dataset_dir, result_base_dir, output_base_dir):
    trackers = ['ocsort']
    
    for tracker in trackers:
        result_dir = os.path.join(result_base_dir, f"{tracker}__input_1280__conf_.25")
        output_dir = os.path.join(output_base_dir, f"{tracker.upper()}__input_1280__conf_.25")
        
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        video_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        
        for video_dir in tqdm(video_dirs, desc=f"Processing videos for {tracker}", unit="video"):
            video_name = video_dir
            input_directory = os.path.join(result_dir, video_name)
            image_directory = os.path.join(dataset_dir, video_name, 'img1')
            process_video(video_name, input_directory, image_directory, output_dir)

# Specify the dataset directory, base result directory, and base output directory
dataset_dir = '/workspace/LiteSORT/datasets/MOT17/train'
result_base_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT17_.25_3_hits'
output_base_dir = '/workspace/LiteSORT/results/off-the-shelf/MOT17_.25_3_hits'

# Run the processing for all videos in the dataset for all trackers
main(dataset_dir, result_base_dir, output_base_dir)


In [ ]:
import os
import re
from PIL import Image
from tqdm import tqdm

def get_image_dimensions(image_directory):
    example_image_path = os.path.join(image_directory, os.listdir(image_directory)[0])
    with Image.open(example_image_path) as img:
        width, height = img.size
    return width, height

def process_file(file_path, image_width, image_height):
    frame_number = os.path.basename(file_path).split('_')[-1].split('.')[0].lstrip('0')  # Remove leading zeros
    frame_number = frame_number if frame_number else '0'  # Ensure frame_number is not empty
    output_data = []
    print(f"Processing file: {file_path}")
    with open(file_path, 'r') as file:
        lines = file.readlines()
        if not lines:
            print(f"File {file_path} is empty.")
        for line in lines:
            data = line.strip().split()
            if len(data) == 6:
                track_id = data[-1]
                bbox_data = data[1:5]  # Ignore the first column
                x_center, y_center, width, height = map(float, bbox_data)
                
                # Convert normalized coordinates to pixel values
                x_center *= image_width
                y_center *= image_height
                width *= image_width
                height *= image_height
                
                # Convert center coordinates to top-left coordinates
                x_top_left = x_center - (width / 2)
                y_top_left = y_center - (height / 2)
                
                output_line = f"{int(frame_number)},{track_id},{x_top_left:.2f},{y_top_left:.2f},{width:.2f},{height:.2f},1,-1,-1,-1"
                output_data.append(output_line)
    print(f"Processed {len(output_data)} lines from file: {file_path}")
    return output_data

def sorted_numerically(file_list):
    def numerical_sort(value):
        numbers = re.findall(r'\d+', value)
        return list(map(int, numbers)) if numbers else [0]
    return sorted(file_list, key=numerical_sort)

def process_video(video_name, input_directory, image_directory, output_directory):
    # Get the dimensions of one image
    image_width, image_height = get_image_dimensions(image_directory)
    
    output_file = os.path.join(output_directory, f"{video_name}.txt")
    
    label_directory = os.path.join(input_directory, 'labels')
    if not os.path.exists(label_directory):
        print(f"Label directory {label_directory} does not exist.")
        return

    files = [file_name for file_name in os.listdir(label_directory) if file_name.endswith('.txt')]
    if not files:
        print(f"No label files found for video {video_name} in directory {label_directory}.")
        return
    
    files = sorted_numerically(files)
    
    all_data = []
    
    for file_name in files:
        file_path = os.path.join(label_directory, file_name)
        output_data = process_file(file_path, image_width, image_height)
        if not output_data:
            print(f"No data processed for file: {file_path}")
        all_data.extend(output_data)
    
    # Sort all data by frame number first, then by track ID
    all_data.sort(key=lambda x: (int(x.split(',')[0]), int(x.split(',')[1])))
    
    with open(output_file, 'w') as out_file:
        for line in all_data:
            out_file.write(line + '\n')
    
    print(f"Data has been written to {output_file}")

def main(dataset_dir, result_base_dir, output_base_dir):
    trackers = ['ocsort']
    
    for tracker in trackers:
        result_dir = os.path.join(result_base_dir, f"{tracker}__input_1280__conf_.25")
        output_dir = os.path.join(output_base_dir, f"{tracker.upper()}__input_1280__conf_.25", "data")
        print(f"result_dir: {result_dir}")
        print(f"output_dir: {output_dir}")
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        video_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        
        for video_dir in tqdm(video_dirs, desc=f"Processing videos for {tracker}", unit="video"):
            video_name = video_dir
            input_directory = os.path.join(result_dir, video_name)
            image_directory = os.path.join(dataset_dir, video_name, 'img1')
            if not os.path.exists(image_directory):
                print(f"Image directory {image_directory} does not exist.")
                continue
            process_video(video_name, input_directory, image_directory, output_dir)

# Specify the dataset directory, base result directory, and base output directory
dataset_dir = '/workspace/LiteSORT/datasets/MOT17/train'
result_base_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT17-train'
output_base_dir = '/workspace/LiteSORT/results/off-the-shelf/MOT17-train'

# Run the processing for all videos in the dataset for all trackers
main(dataset_dir, result_base_dir, output_base_dir)


#### KITTI

not formatted output

In [ ]:
import os
import cv2

def draw_bounding_boxes(image_dir, result_file, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    with open(result_file, 'r') as file:
        lines = file.readlines()
    
    bbox_data = {}
    for line in lines:
        data = line.strip().split(',')
        frame_id = int(data[0])
        track_id = int(data[1])
        x, y, w, h = map(float, data[2:6])
        
        if frame_id not in bbox_data:
            bbox_data[frame_id] = []
        
        bbox_data[frame_id].append((track_id, x, y, w, h))
    
    for frame_id in bbox_data:
        image_path = os.path.join(image_dir, f"{frame_id:06d}.png")  # Update the extension to .png
        # print(f"Checking if file is accessible: {image_path}")
        if not os.access(image_path, os.R_OK):
            print(f"Error: No read access to file {image_path}")
            continue
        
        print(f"Reading image from path: {image_path}")
        image = cv2.imread(image_path)
        
        if image is None:
            print(f"Error: Image at path {image_path} could not be read. Check the file path and permissions.")
            continue
        
        for bbox in bbox_data[frame_id]:
            track_id, x, y, w, h = bbox
            top_left = (int(x), int(y))
            bottom_right = (int(x + w), int(y + h))
            cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
            cv2.putText(image, str(track_id), (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        output_image_path = os.path.join(output_dir, f"{frame_id:06d}.png")  # Update the extension to .png
        cv2.imwrite(output_image_path, image)
        print(f"Processed frame {frame_id}")


seq = '0005'

# Example usage
image_dir = f'/media/hbai/data/code/LiteSORT/datasets/KITTI/train/{seq}/img1'  # Adjust this to your image directory
result_file = f'/media/hbai/data/code/LiteSORT/results/off-the-shelf/KITTI-train_fixed-settings_not-formatted/BoTSORT__input_1280__conf_.25/data/{seq}.txt'  # Adjust this to your result file
output_dir = f'/media/hbai/data/code/LiteSORT/results/off-the-shelf/KITTI-train_fixed-settings_not-formatted/viz_BoTSORT__input_1280__conf_.25/{seq}'  # Adjust this to your desired output directory

draw_bounding_boxes(image_dir, result_file, output_dir)


formatted output

In [ ]:
import os
import cv2

def draw_bounding_boxes(image_dir, result_file, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    with open(result_file, 'r') as file:
        lines = file.readlines()
    
    bbox_data = {}
    for line in lines:
        data = line.strip().split()
        frame_id = int(data[0])
        track_id = int(data[1])
        left = float(data[6])
        top = float(data[7])
        right = float(data[8])
        bottom = float(data[9])
        
        if frame_id not in bbox_data:
            bbox_data[frame_id] = []
        
        bbox_data[frame_id].append((track_id, left, top, right, bottom))
    
    for frame_id in bbox_data:
        image_path = os.path.join(image_dir, f"{frame_id:06d}.png")
        # print(f"Checking if file is accessible: {image_path}")
        if not os.access(image_path, os.R_OK):
            print(f"Error: No read access to file {image_path}")
            continue
        
        print(f"Reading image from path: {image_path}")
        image = cv2.imread(image_path)
        
        if image is None:
            print(f"Error: Image at path {image_path} could not be read. Check the file path and permissions.")
            continue
        
        for bbox in bbox_data[frame_id]:
            track_id, left, top, right, bottom = bbox
            top_left = (int(left), int(top))
            bottom_right = (int(right), int(bottom))
            cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
            cv2.putText(image, str(track_id), (int(left), int(top) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        output_image_path = os.path.join(output_dir, f"{frame_id:06d}.png")
        cv2.imwrite(output_image_path, image)
        print(f"Processed frame {frame_id}")

seq = '0006'

# Example usage
image_dir = f'/media/hbai/data/code/LiteSORT/datasets/KITTI/train/{seq}/img1'  # Adjust this to your image directory
result_file = f'/media/hbai/data/code/LiteSORT/results/off-the-shelf/fixed-settings/KITTI-train_fixed_settings/BoTSORT__input_1280__conf_.25/data/{seq}.txt'  # Adjust this to your result file
output_dir = f'/media/hbai/data/code/LiteSORT/results/off-the-shelf/fixed-settings/KITTI-train_fixed_settings/viz_BoTSORT__input_1280__conf_.25/{seq}'  # Adjust this to your desired output directory

draw_bounding_boxes(image_dir, result_file, output_dir)


#### DanceTrack

In [1]:
import os
import cv2

def draw_bounding_boxes(image_dir, result_file, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    with open(result_file, 'r') as file:
        lines = file.readlines()
    
    bbox_data = {}
    for line in lines:
        data = line.strip().split(',')
        frame_id = int(data[0])
        track_id = int(data[1])
        x, y, w, h = map(float, data[2:6])
        
        if frame_id not in bbox_data:
            bbox_data[frame_id] = []
        
        bbox_data[frame_id].append((track_id, x, y, w, h))
    
    for frame_id in bbox_data:
        image_path = os.path.join(image_dir, f"{frame_id:08d}.jpg")
        image = cv2.imread(image_path)
        
        for bbox in bbox_data[frame_id]:
            track_id, x, y, w, h = bbox
            top_left = (int(x), int(y))
            bottom_right = (int(x + w), int(y + h))
            cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
            cv2.putText(image, str(track_id), (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        output_image_path = os.path.join(output_dir, f"{frame_id:08d}.jpg")
        
        cv2.imwrite(output_image_path, image)
        print(f"Processed frame {frame_id}")

# Example usage
image_dir = '/media/hbai/data/code/LiteSORT/datasets/DanceTrack/train/dancetrack0001/img1'  # Adjust this to your image directory
result_file = '/media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/deepocsort__input_1280__conf_.25__IoU_0.2/data/dancetrack0001.txt'  # Adjust this to your result file
output_dir = '/media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/viz_fixed_deepocsort__input_1280__conf_.25__IoU_0.2/data/dancetrack0001'  # Adjust this to your desired output directory

draw_bounding_boxes(image_dir, result_file, output_dir)


Processed frame 4
Processed frame 5
Processed frame 6
Processed frame 7
Processed frame 8
Processed frame 9
Processed frame 10
Processed frame 11
Processed frame 12
Processed frame 13
Processed frame 14
Processed frame 15
Processed frame 16
Processed frame 17
Processed frame 18
Processed frame 19
Processed frame 20
Processed frame 21
Processed frame 22
Processed frame 23
Processed frame 24
Processed frame 25
Processed frame 26
Processed frame 27
Processed frame 28
Processed frame 29
Processed frame 30
Processed frame 31
Processed frame 32
Processed frame 33
Processed frame 34
Processed frame 35
Processed frame 36
Processed frame 37
Processed frame 38
Processed frame 39
Processed frame 40
Processed frame 41
Processed frame 42
Processed frame 43
Processed frame 44
Processed frame 45
Processed frame 46
Processed frame 47
Processed frame 48
Processed frame 49
Processed frame 50
Processed frame 51
Processed frame 52
Processed frame 53
Processed frame 54
Processed frame 55
Processed frame 56